# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [126]:
!pip install citipy

In [127]:
# Dependencies
import json
import requests
import pandas as pd
from api_keys import weather_api_key, g_key
from citipy import citipy
import numpy as np
from scipy.stats import linregress
from scipy import stats
from pprint import pprint
import os
import datetime as dt
import time

#file; for the end
data = "../Output/cities.csv"



## Generate Cities List

In [128]:
# Ranges of coords
range_lat = (-90, 90)
range_lng = (-180, 180)

# List for holding coords and cities
coordinates = []
cities = []

#gather random coords
np.random.seed(1000)

lats = np.random.uniform(range_lat[0], range_lat[1], size=2000)
longs = lngs = np.random.uniform(range_lng[0], range_lng[1], size=2000)

coordinates = zip(lats, longs)

#Nearest city
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

# If the city is unique, then add it to cities list
    if city not in cities:
        cities.append(city)
        


In [129]:
# Print the city count to confirm sufficient count
len(cities)

759

In [130]:
# Convert city list to dataframe and rename column to "city"
weather_dict = {
    "City": cities,
}
weather_data = pd.DataFrame(weather_dict)

# Add columns to cities dataframe
weather_data["Cloudiness"] = np.nan
weather_data["Country"] = np.nan
weather_data["Date"] = np.nan
weather_data["Humidity"] = np.nan
weather_data["Lat"] = np.nan
weather_data["Lng"] = np.nan
weather_data["Max Temp"] = np.nan
weather_data["Wind Speed"] = np.nan
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,dali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bluff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,norman wells,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tabiauea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,leningradskiy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [131]:
# Save config information #reference
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

city = "London"

# Build query URL | F-strings can be used too.
# query_url = f"{url}""
query_url = url + "appid=" + weather_api_key + "&units=" + units + "&q=" +city


# Get weather data
weather_response = requests.get(query_url)
#convert to a json
weather_json = weather_response.json()

# Get the temperature from the response
print(f"The weather API responded with: {weather_json}.")

The weather API responded with: {'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 74.23, 'feels_like': 73.56, 'temp_min': 66.74, 'temp_max': 77.18, 'pressure': 1013, 'humidity': 47}, 'visibility': 10000, 'wind': {'speed': 3, 'deg': 92, 'gust': 8.99}, 'clouds': {'all': 85}, 'dt': 1622655152, 'sys': {'type': 2, 'id': 268730, 'country': 'GB', 'sunrise': 1622605698, 'sunset': 1622664543}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}.


In [132]:
weather_json

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 74.23,
  'feels_like': 73.56,
  'temp_min': 66.74,
  'temp_max': 77.18,
  'pressure': 1013,
  'humidity': 47},
 'visibility': 10000,
 'wind': {'speed': 3, 'deg': 92, 'gust': 8.99},
 'clouds': {'all': 85},
 'dt': 1622655152,
 'sys': {'type': 2,
  'id': 268730,
  'country': 'GB',
  'sunrise': 1622605698,
  'sunset': 1622664543},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [133]:
weather_json['coord']['lon']
weather_json['coord']['lat']
weather_json['clouds']['all']
weather_json['sys']['country']
weather_json['main']['humidity']
weather_json['main']['temp_max']
weather_json['wind']['speed']
# dt.date(weather_json['dt'])

-0.1257

51.5085

85

'GB'

47

77.18

3

In [134]:
dt.datetime.fromtimestamp(int(weather_json['dt'])).strftime('%Y-%m-%d %H:%M:%S')

'2021-06-02 12:32:32'

In [135]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"

cities_found = []
cloudiness = []
countries = []
dates = []
humidities = []
lats = []
lngs = []
max_temps = []
wind_speeds = []

# Use iterrows to loop through dataframe using the city name
for city in cities:
    time.sleep(1)
    # Build query URL | F-strings can be used too.
    # query_url = f"{url}""
    query_url = url + "appid=" + weather_api_key + "&q=" + city.replace(" ","+")

    # Get weather data
    weather_response = requests.get(query_url)
    #convert to a json
    weather_json = weather_response.json()
    try:
        dates.append(dt.datetime.fromtimestamp(int(weather_json['dt'])).strftime('%Y-%m-%d %H:%M:%S'))
        cloudiness.append(weather_json['clouds']['all'])
        countries.append(weather_json['sys']['country'])
        humidities.append(weather_json['main']['humidity'])
        lats.append(weather_json['coord']['lat'])
        lngs.append(weather_json['coord']['lon'])
        max_temps.append(weather_json['main']['temp_max'])
        wind_speeds.append(weather_json['wind']['speed'])
        cities_found.append(city)
        
        print(f"✅{city} data processing")
    
    except:
        print(f"❌{city} not found")

  
    
    


✅dali data processing
✅bluff data processing
✅norman wells data processing
❌tabiauea not found
✅leningradskiy data processing
✅tuatapere data processing
✅kaitangata data processing
✅avarua data processing
✅vaini data processing
✅okhotsk data processing
✅arraial do cabo data processing
✅chifeng data processing
✅hithadhoo data processing
❌tsihombe not found
❌tasbuget not found
✅hermanus data processing
✅sorland data processing
✅honningsvag data processing
✅talnakh data processing
✅georgetown data processing
✅port alfred data processing
✅barrow data processing
✅rikitea data processing
✅seoul data processing
✅punta arenas data processing
✅oktyabrskoye data processing
✅hasaki data processing
✅atasu data processing
❌taburi not found
✅ilulissat data processing
✅tromso data processing
✅ushuaia data processing
✅mahebourg data processing
❌aybak not found
✅zhigansk data processing
✅saint-philippe data processing
✅atuona data processing
✅nikolskoye data processing
✅san cristobal data processing
✅b

✅ribeira grande data processing
❌palabuhanratu not found
✅okha data processing
✅maningrida data processing
✅komsomolskiy data processing
✅camopi data processing
❌ituni not found
✅kungurtug data processing
✅rio verde de mato grosso data processing
✅sandnessjoen data processing
✅olafsvik data processing
✅kidal data processing
✅samarai data processing
✅nome data processing
✅shwebo data processing
❌nguiu not found
✅namatanai data processing
✅ambatofinandrahana data processing
✅anaconda data processing
✅abhar data processing
✅darlawn data processing
✅paamiut data processing
✅dondo data processing
✅boa vista data processing
❌marcona not found
✅anadyr data processing
✅pandaan data processing
✅pimenta bueno data processing
✅abu dhabi data processing
✅kichera data processing
✅urussu data processing
✅cabatuan data processing
✅leona vicario data processing
❌turtkul not found
✅mehamn data processing
✅dobre miasto data processing
✅prado data processing
✅tezu data processing
✅zabol data processing
✅

✅voykove data processing
✅mananjary data processing
✅alnashi data processing
✅babaita data processing
✅houma data processing
✅bonavista data processing
✅sampit data processing
✅nandyal data processing
✅boyolangu data processing
✅balkanabat data processing
✅deputatskiy data processing
❌jabiru not found
✅staryy nadym data processing
✅mahina data processing
✅teguise data processing
✅mackay data processing
✅moose factory data processing
✅watford data processing
✅capitan bado data processing
✅riachao das neves data processing
✅guatire data processing
✅caiaponia data processing
✅ballinasloe data processing
✅itapecerica data processing
✅poole data processing
✅xiamen data processing
✅batagay-alyta data processing
✅uige data processing
✅malinovskiy data processing
✅santa marta data processing
✅haaksbergen data processing
✅burns lake data processing
❌beypur not found
❌ngukurr not found
✅kabalo data processing
✅kumluca data processing
✅karratha data processing
✅saquena data processing
✅ujiie data

In [143]:
#sum of countries
len(countries)

700

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [137]:
# cities_found = []
# cloudiness = []
# countries = []
# dates = []
# humidities = []
# lats = []
# lngs = []
# max_temps = []
# wind_speeds = []

# Convert city list to dataframe and rename column to "city"

weather_data = pd.DataFrame({
    "Date": dates,
    "City": cities_found,
    "Country": countries,
    "Cloudiness": cloudiness,
    "Humidity": humidities,
    "Latitude": lats,
    "Longitude": lngs,
    "Maximum Temp": max_temps,
    "Wind Speed": wind_speeds
    
})

weather_data

,Date,City,Country,Cloudiness,Humidity,Latitude,Longitude,Maximum Temp,Wind Speed
0,2021-06-02 12:32:59,dali,CN,37,86,25.7000,100.1833,291.24,2.57
1,2021-06-02 12:33:01,bluff,NZ,71,81,-46.6000,168.3333,283.04,4.65
2,2021-06-02 12:33:02,norman wells,CA,75,76,65.2820,-126.8329,280.17,4.63
3,2021-06-02 12:33:05,leningradskiy,RU,59,94,69.3833,178.4167,269.84,4.36
4,2021-06-02 12:33:06,tuatapere,NZ,30,89,-46.1333,167.6833,283.05,2.35
...,...,...,...,...,...,...,...,...,...
695,2021-06-02 12:50:21,dunedin,NZ,1,88,-45.8742,170.5036,280.82,0.45
696,2021-06-02 12:50:22,saint-joseph,RE,95,79,-21.3667,55.6167,297.36,6.44
697,2021-06-02 12:50:24,ekhabi,RU,59,95,53.5102,142.9671,279.46,2.91
698,2021-06-02 12:50:25,itupeva,BR,0,39,-23.1531,-47.0578,300.00,2.51


In [138]:
weather_data.to_csv("WeatherPyData.csv")

In [139]:
weather_data.count()

Date            700
City            700
Country         700
Cloudiness      700
Humidity        700
Latitude        700
Longitude       700
Maximum Temp    700
Wind Speed      700
dtype: int64

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [140]:
weather_data[weather_data["Humidity"]>100]

,Date,City,Country,Cloudiness,Humidity,Latitude,Longitude,Maximum Temp,Wind Speed


In [141]:
#  Get the indices of cities that have humidity over 100%.
#no cities found with humidity over 100 - skipping

In [142]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression